# Operando con datos en Pandas

Una de las piezas esenciales de NumPy es la capacidad de realizar operaciones rápidas entre elementos, tanto con aritmética básica (suma, resta, multiplicación, etc.) como con operaciones más sofisticadas (funciones trigonométricas, funciones exponenciales y logarítmicas, etc.).
**Pandas hereda gran parte de esta funcionalidad de NumPy y las ufuncs** son claves para esto.

Sin embargo, Pandas incluye un par de cambios útiles: para operaciones unarias como negación y funciones trigonométricas, estas ufuncs *preservarán las etiquetas de índice y columna* en la salida, y para operaciones binarias como **suma y multiplicación, Pandas *alineará automáticamente los índices *al pasar los objetos al ufunc.**
Esto significa que mantener el contexto de los datos y combinar datos de diferentes fuentes (ambas tareas potencialmente propensas a errores con matrices NumPy sin procesar) se vuelven esencialmente infalibles con Pandas.
Veremos adicionalmente que existen operaciones bien definidas entre estructuras unidimensionales ``Series`` y estructuras bidimensionales ``DataFrame``.

## Ufuncs: Preservación del índice

Debido a que Pandas está diseñado para funcionar con NumPy, cualquier ufunc de NumPy funcionará en los objetos ``Series`` y ``DataFrame`` de Pandas.
Comencemos definiendo una ``Serie`` y un ``DataFrame`` simples para demostrar esto:

In [1]:
import pandas as pd
import numpy as np

In [6]:
rng = np.random.RandomState(42) #controlo aleatoriedad
ser = pd.Series(rng.randint(0, 10, 4))
ser

0    6
1    3
2    7
3    4
dtype: int32

In [3]:
df = pd.DataFrame(rng.randint(0, 10, (3, 4)),
                  columns=['A', 'B', 'C', 'D'])
df

,A,B,C,D
0,6,9,2,6
1,7,4,3,7
2,7,2,5,4


Si aplicamos una ufunc NumPy en cualquiera de estos objetos, el resultado será otro objeto Pandas *con los índices conservados:*

In [4]:
np.exp(ser)

0     403.428793
1      20.085537
2    1096.633158
3      54.598150
dtype: float64

O, para un cálculo un poco más complejo:

In [5]:
np.sin(df * np.pi / 4)

,A,B,C,D
0,-1.000000,7.071068e-01,1.000000,-1.000000e+00
1,-0.707107,1.224647e-16,0.707107,-7.071068e-01
2,-0.707107,1.000000e+00,-0.707107,1.224647e-16


## UFuncs: Alineación del índice

Para operaciones binarias en dos objetos ``Series`` o ``DataFrame``, Pandas alineará los índices en el proceso de realizar la operación.
Esto es muy conveniente cuando se trabaja con datos incompletos, como veremos en algunos de los ejemplos siguientes.

### Alineación de índice en serie

Como ejemplo, supongamos que combinamos dos fuentes de datos diferentes y encontramos solo los tres principales estados de EE. UU. por *área* y los tres principales estados de EE. UU. por *población*:

In [7]:
area = pd.Series({'Alaska': 1723337,
                  'Texas': 695662,
                  'California': 423967},
                 name='area')

population = pd.Series({'California': 38332521,
                        'Texas': 26448193,
                        'New York': 19651127},
                       name='population')

Veamos qué sucede cuando los dividimos para calcular la densidad de población:

In [8]:
type(population / area)

pandas.core.series.Series

In [9]:
s_poblacion_area = population / area
print(s_poblacion_area)

Alaska              NaN
California    90.413926
New York            NaN
Texas         38.018740
dtype: float64


La matriz resultante contiene la *unión* de índices de las dos matrices de entrada, que podrían determinarse usando la aritmética de conjuntos estándar de Python en estos índices:

In [10]:
area.index

Index(['Alaska', 'Texas', 'California'], dtype='object')

In [16]:
population.index

Index(['California', 'Texas', 'New York'], dtype='object')

In [13]:
set(area.index) | set(population.index)

{'Alaska', 'California', 'New York', 'Texas'}

In [14]:
area.index.union(population.index)

Index(['Alaska', 'California', 'New York', 'Texas'], dtype='object')

Cualquier elemento para el cual uno u otro no tenga una entrada se marca con ``NaN``, o "No es un número", que es como Pandas marca los datos faltantes.
Esta coincidencia de índice se implementa de esta manera para cualquiera de las expresiones aritméticas integradas de Python; cualquier valor faltante se completa con NaN de forma predeterminada:

In [17]:
A = pd.Series([2, 4, 6], index=["andalucia", "aragon", "madrid"])
print(A)
B = pd.Series([1, 3, 5], index=["aragon", "madrid", "asturias"])
print(B)
A + B

andalucia    2
aragon       4
madrid       6
dtype: int64
aragon      1
madrid      3
asturias    5
dtype: int64


andalucia    NaN
aragon       5.0
asturias     NaN
madrid       9.0
dtype: float64

In [18]:
df_AB = pd.DataFrame({'n_alumnos': A, 'n_profesores': B})
df_AB #NaN sale cuando no hay un dato: numero no informado

,n_alumnos,n_profesores
andalucia,2.0,NaN
aragon,4.0,1.0
asturias,NaN,5.0
madrid,6.0,3.0


In [19]:
# df_AB = pd.DataFrame({'n_alumnos': A, 'n_profesores': B})
df_AB['n_personas'] = df_AB['n_alumnos'] + df_AB['n_profesores']
df_AB

,n_alumnos,n_profesores,n_personas
andalucia,2.0,NaN,NaN
aragon,4.0,1.0,5.0
asturias,NaN,5.0,NaN
madrid,6.0,3.0,9.0


In [20]:
df_AB['ratio_a'] = df_AB['n_alumnos'] / df_AB['n_profesores']
df_AB

,n_alumnos,n_profesores,n_personas,ratio_a
andalucia,2.0,NaN,NaN,NaN
aragon,4.0,1.0,5.0,4.0
asturias,NaN,5.0,NaN,NaN
madrid,6.0,3.0,9.0,2.0


In [24]:
df_AB['n_personas'].fillna(0) #intercambio el NaN por 0 una vez hecha la suma

andalucia    0.0
aragon       5.0
asturias     0.0
madrid       9.0
Name: n_personas, dtype: float64

Si usar valores NaN no es el comportamiento deseado, el valor de relleno se puede modificar usando métodos de objeto apropiados en lugar de los operadores.
Por ejemplo, llamar a ``A.add(B)`` es equivalente a llamar a ``A + B``, pero permite la especificación explícita opcional del valor de relleno para cualquier elemento en ``A`` o ``B`` que podría faltar:

In [23]:
serie_1 = A.add(B, fill_value=0) #fill_value=0 me rellena datos ausentes a 0 antes de hacer la suma
print(serie_1)

andalucia    2.0
aragon       5.0
asturias     5.0
madrid       9.0
dtype: float64


### Alineación de índice en DataFrame

Un tipo similar de alineación tiene lugar para *ambas* columnas e índices cuando se realizan operaciones en ``DataFrame``s:

In [32]:
df_A = pd.DataFrame(rng.randint(0, 20, (2, 2)),
                 columns=list('AB'))
df_A

,A,B
0,11,11
1,16,9


In [34]:
B = pd.DataFrame(rng.randint(0, 10, (3, 3)),
                 columns=list('BAC'))
B

,B,A,C
0,2,6,3
1,8,2,4
2,2,6,4


In [36]:
df_A + B

,A,B,C
0,17.0,13.0,NaN
1,18.0,17.0,NaN
2,NaN,NaN,NaN


In [37]:
df_A.add(B, fill_value=0)

,A,B,C
0,17.0,13.0,3.0
1,18.0,17.0,4.0
2,6.0,2.0,4.0


Observe que los índices están alineados correctamente independientemente de su orden en los dos objetos y los índices del resultado están ordenados.
Como fue el caso con ``Series``, podemos usar el método aritmético del objeto asociado y pasar cualquier ``fill_value`` que deseemos para usar en lugar de las entradas faltantes.
Aquí completaremos con la media de todos los valores en ``A`` (calculada apilando primero las filas de ``A``):

In [38]:
df_A.values.mean()

11.75

In [43]:
fill = df_A.values.mean()
df_A.add(B, fill_value=fill)
#al no ser dos dataframes de las mismas dimensiones me
#saldria NaN, por lo que debo aplicar fill_value

,A,B,C
0,17.00,13.00,14.75
1,18.00,17.00,15.75
2,17.75,13.75,15.75


La siguiente tabla enumera los operadores de Python y sus métodos de objeto Pandas equivalentes:

| Python Operator | Pandas Method(s)                      |
|-----------------|---------------------------------------|
| ``+``           | ``add()``                             |
| ``-``           | ``sub()``, ``subtract()``             |
| ``*``           | ``mul()``, ``multiply()``             |
| ``/``           | ``truediv()``, ``div()``, ``divide()``|
| ``//``          | ``floordiv()``                        |
| ``%``           | ``mod()``                             |
| ``**``          | ``pow()``                             |


## Ufuncs: Operaciones entre DataFrame y Series

Al realizar operaciones entre un ``DataFrame`` y una ``Series``, la alineación del índice y la columna se mantiene de manera similar.
Las operaciones entre un ``DataFrame`` y una ``Series`` son similares a las operaciones entre una matriz NumPy bidimensional y unidimensional.
Considere una operación común, donde encontramos la diferencia entre una matriz bidimensional y una de sus filas:

In [44]:
A = rng.randint(10, size=(3, 4))
A

array([[8, 6, 1, 3],
       [8, 1, 9, 8],
       [9, 4, 1, 3]])

In [45]:
A - A[0] #se hace la resta a nivel de filas

array([[ 0,  0,  0,  0],
       [ 0, -5,  8,  5],
       [ 1, -2,  0,  0]])

Según las reglas de transmisión de NumPy, la resta entre una matriz bidimensional y una de sus filas se aplica fila por fila.

En Pandas, la convención opera de manera similar en filas de forma predeterminada:

In [46]:
df = pd.DataFrame(A, columns=list('QRST'))
print(df)
df - df.iloc[0]

   Q  R  S  T
0  8  6  1  3
1  8  1  9  8
2  9  4  1  3


,Q,R,S,T
0,0,0,0,0
1,0,-5,8,5
2,1,-2,0,0


Si, en cambio, desea operar en columnas, puede utilizar los métodos de objeto mencionados anteriormente, mientras especifica la palabra clave ``axis``:

In [48]:
df = df.subtract(df['R'], axis=0) #axis=0 quiere decir a nivel columnar

In [49]:
df

,Q,R,S,T
0,2,0,-5,-3
1,7,0,8,7
2,5,0,-3,-1


Tenga en cuenta que estas operaciones ``DataFrame``/``Series``, como las operaciones analizadas anteriormente, alinearán automáticamente los índices entre los dos elementos:

In [50]:
df

,Q,R,S,T
0,2,0,-5,-3
1,7,0,8,7
2,5,0,-3,-1


In [51]:
halfrow = df.iloc[0, ::2]
halfrow

Q    2
S   -5
Name: 0, dtype: int32

In [52]:
df - halfrow

,Q,R,S,T
0,0.0,NaN,0.0,NaN
1,5.0,NaN,13.0,NaN
2,3.0,NaN,2.0,NaN


In [53]:
df.subtract(halfrow)

,Q,R,S,T
0,0.0,NaN,0.0,NaN
1,5.0,NaN,13.0,NaN
2,3.0,NaN,2.0,NaN


Esta preservación y alineación de índices y columnas significa que las operaciones con datos en Pandas siempre mantendrán el contexto de los datos, lo que evita los tipos de errores tontos que pueden surgir al trabajar con datos heterogéneos y/o desalineados en matrices NumPy sin procesar.